In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./Free_Adversarial_Training.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./Free_Adversarial_Training.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def4/y_pred_{attack_name}{eps}_Def4.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
def fgsm(grad, epsilon):
    sign_grad = grad.sign()
    return epsilon * sign_grad
    

In [11]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

# Adversarial training parameters
epsilon = 0.03  # Step size for FGSM
clip_eps = 0.1  # Maximum perturbation
n_repeats = 4   # Number of iterations to update noise

global_noise_data = torch.zeros(train_loader.batch_size, input_shape).to(device)


In [12]:
# from torch.autograd import Variable


# for epoch in range(10):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         for j in range(n_repeats):
#             noise_batch = Variable(global_noise_data[0:inputs.size(0)], requires_grad=True).to(device)
#             inputs_adv = inputs + noise_batch
#             inputs_adv.clamp_(0, 1.0)
#             optimizer.zero_grad()
#             outputs = model(inputs_adv)
#             loss = loss_function(outputs, labels)
#             loss.backward()
#             pert = fgsm(noise_batch.grad, epsilon)
#             global_noise_data[0:inputs.size(0)] += pert.data
#             global_noise_data.clamp_(-clip_eps, clip_eps)
#             optimizer.step()
#             train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [13]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Free_Adversarial_Training/Free_Adversarial_Training.pt"))

/tmp/ipykernel_3541/1052584200.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Fr

<All keys matched successfully>

In [14]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [15]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./Free_Adversarial_Training.csv", mode='a', index=False, header=False)

In [16]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [17]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(853, 56) (853,)
Accuracy: 0.7069167643610785

macro
Precision: 0.6003510716925351
Recall: 0.5180556663741057
F1 Score: 0.46895668977607774

weighted
Precision: 0.6484207297577612
Recall: 0.7069167643610785
F1 Score: 0.6169003438525669

start RF 10 BIM
(518, 56) (518,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8957528957528957

macro
Precision: 0.5
Recall: 0.44787644787644787
F1 Score: 0.4725050916496945

weighted
Precision: 1.0
Recall: 0.8957528957528957
F1 Score: 0.945010183299389

start RF 10 FGSM
(393, 56) (393,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(518, 56) (518,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8957528957528957

macro
Precision: 0.5
Recall: 0.44787644787644787
F1 Score: 0.4725050916496945

weighted
Precision: 1.0
Recall: 0.8957528957528957
F1 Score: 0.945010183299389

start RF 10 DF
(189, 56) (189,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9206349206349206

macro
Precision: 0.5
Recall: 0.4603174603174603
F1 Score: 0.4793388429752066

weighted
Precision: 1.0
Recall: 0.9206349206349206
F1 Score: 0.9586776859504131

start RF 10 AutoPGD
(1898, 56) (1898,)
Accuracy: 0.946786090621707

macro
Precision: 0.5447574859783479
Recall: 0.6285867237687366
F1 Score: 0.5618962298000516

weighted
Precision: 0.9743671228007282
Recall: 0.946786090621707
F1 Score: 0.9595508471076417

start RF 10 ZOO
(2319, 56) (2319,)
Accuracy: 0.7705907718844329

macro
Precision: 0.6134197199880846
Recall: 0.5218889863601716
F1 Score: 0.49366313487672664

weighted
Precision: 0.7061776008708158
Recall: 0.7705907718844329
F1 Score: 0.6992193342416305

start RF 10 CaFA
(591, 56) (591,)
Accuracy: 0.7258883248730964

macro
Precision: 0.7353475810849411
Recall: 0.7099666388657214
F1 Score: 0.7112656517648194

weighted
Precision: 0.7324119498467825
Recall: 0.7258883248730964
F1 Score: 0.7181921811318979

start RF 10 SINIFGSM
(326, 56) (326,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9815950920245399

macro
Precision: 0.5
Recall: 0.49079754601226994
F1 Score: 0.4953560371517028

weighted
Precision: 1.0
Recall: 0.9815950920245399
F1 Score: 0.9907120743034055

start RF 10 VNIFGSM
(355, 56) (355,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0



In [18]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(889, 56) (889,)
Accuracy: 0.6872890888638921

macro
Precision: 0.6164980732177263
Recall: 0.5141830870279146
F1 Score: 0.448461936727844

weighted
Precision: 0.6441915858783548
Recall: 0.6872890888638921
F1 Score: 0.5827766940829889

start RF 10 BIM
(376, 56) (376,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9654255319148937

macro
Precision: 0.5
Recall: 0.48271276595744683
F1 Score: 0.4912043301759134

weighted
Precision: 1.0
Recall: 0.9654255319148937
F1 Score: 0.9824086603518268

start RF 10 FGSM
(338, 56) (338,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(376, 56) (376,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9654255319148937

macro
Precision: 0.5
Recall: 0.48271276595744683
F1 Score: 0.4912043301759134

weighted
Precision: 1.0
Recall: 0.9654255319148937
F1 Score: 0.9824086603518268

start RF 10 DF
(212, 56) (212,)
Accuracy: 0.8160377358490566

macro
Precision: 0.524390243902439
Recall: 0.9071428571428571
F1 Score: 0.49533052554477197

weighted
Precision: 0.9910262310170271
Recall: 0.8160377358490566
F1 Score: 0.8900470920731222

start RF 10 AutoPGD
(1887, 56) (1887,)
Accuracy: 0.9523052464228935

macro
Precision: 0.48384491114701134
Recall: 0.4917898193760263
F1 Score: 0.48778501628664495

weighted
Precision: 0.9369206705517644
Recall: 0.9523052464228935
F1 Score: 0.9445503177060947

start RF 10 ZOO
(2288, 56) (2288,)
Accuracy: 0.7482517482517482

macro
Precision: 0.6595803044849815
Recall: 0.5245253474261108
F1 Score: 0.4862011050344095

weighted
Precision: 0.7056894552437002
Recall: 0.7482517482517482
F1 Score: 0.665498913551536

start RF 10 CaFA
(177, 56) (177,)
Accuracy: 0.

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9904458598726115

macro
Precision: 0.5
Recall: 0.49522292993630573
F1 Score: 0.4976

weighted
Precision: 1.0
Recall: 0.9904458598726115
F1 Score: 0.9952

start RF 10 VNIFGSM
(352, 56) (352,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0



In [19]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start RF 10 baseline
(1216, 56) (1216,)
Accuracy: 0.7138157894736842

macro
Precision: 0.58428957383145
Recall: 0.5096557347044044
F1 Score: 0.4502112533325018

weighted
Precision: 0.6431244524392351
Recall: 0.7138157894736842
F1 Score: 0.6148858210406514

start RF 10 BIM
(100, 56) (100,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.68

macro
Precision: 0.5
Recall: 0.34
F1 Score: 0.40476190476190477

weighted
Precision: 1.0
Recall: 0.68
F1 Score: 0.8095238095238095

start RF 10 FGSM
(54, 56) (54,)
Accuracy: 0.9444444444444444

macro
Precision: 0.49038461538461536
Recall: 0.4811320754716981
F1 Score: 0.4857142857142857

weighted
Precision: 0.9626068376068375
Recall: 0.9444444444444444
F1 Score: 0.9534391534391534

start RF 10 PGD
(100, 56) (100,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.68

macro
Precision: 0.5
Recall: 0.34
F1 Score: 0.40476190476190477

weighted
Precision: 1.0
Recall: 0.68
F1 Score: 0.8095238095238095

start RF 10 DF
(195, 56) (195,)
Accuracy: 0.8974358974358975

macro
Precision: 0.602881698685541
Recall: 0.8092705167173253
F1 Score: 0.6388888888888888

weighted
Precision: 0.9606958595836249
Recall: 0.8974358974358975
F1 Score: 0.9225071225071225

start RF 10 AutoPGD
(2567, 56) (2567,)
Accuracy: 0.9333852746396571

macro
Precision: 0.4793917567026811
Recall: 0.4862012987012987
F1 Score: 0.4827725166230103

weighted
Precision: 0.9203126517455444
Recall: 0.9333852746396571
F1 Score: 0.9268028679073606

start RF 10 ZOO
(3225, 56) (3225,)
Accuracy: 0.7686821705426357

macro
Precision: 0.6135902745929833
Recall: 0.5172897751813863
F1 Score: 0.48210324827184775

weighted
Precision: 0.7027159423999545
Recall: 0.7686821705426357
F1 Score: 0.691273607961781

start RF 10 CaFA
(185, 56) (185,)
Accuracy: 0.34054054054054056

macro
Precision: 0.512113

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6611661166116611

macro
Precision: 0.5418247902079214
Recall: 0.5053225716489566
F1 Score: 0.42983870967741933

weighted
Precision: 0.5839288562989604
Recall: 0.6611661166116611
F1 Score: 0.5504965281474383

start RF 10 BIM
(2684, 56) (2684,)
Accuracy: 0.5432190760059612

macro
Precision: 0.5637264395603907
Recall: 0.7348169171205459
F1 Score: 0.45458113629869196

weighted
Precision: 0.9309364182068918
Recall: 0.5432190760059612
F1 Score: 0.6423428348283234

start RF 10 FGSM
(4093, 56) (4093,)
Accuracy: 0.7730271194722698

macro
Precision: 0.6268608841971224
Recall: 0.8469966958211856
F1 Score: 0.6341092544238857

weighted
Precision: 0.9310688033067295
Recall: 0.7730271194722698
F1 Score: 0.8217750618068533

start RF 10 PGD
(2684, 56) (2684,)
Accuracy: 0.5432190760059612

macro
Precision: 0.5637264395603907
Recall: 0.7348169171205459
F1 Score: 0.45458113629869196

weighted
Precision: 0.9309364182068918
Recall: 0.5432190760059612
F1 Score: 0.6423428348283234

start RF 10 DF


In [16]:
epsilon = 0

Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9138321995464853

macro
Precision: 0.8151260504201681
Recall: 0.5400041562759768
F1 Score: 0.5519357828690941

weighted
Precision: 0.8976733550563083
Recall: 0.9138321995464853
F1 Score: 0.8815689361589297

start BIM
Accuracy: 0.6554959785522788

macro
Precision: 0.5327906150017775
Recall: 0.5996347589639599
F1 Score: 0.4908733656042393

weighted
Precision: 0.8748318158988
Recall: 0.6554959785522788
F1 Score: 0.7331891589204955

start FGSM
Accuracy: 0.7620614035087719

macro
Precision: 0.5517007750971163
Recall: 0.6456794972505892
F1 Score: 0.5444449417683446

weighted
Precision: 0.8992531176902864
Recall: 0.7620614035087719
F1 Score: 0.8155736481990409

start PGD
Accuracy: 0.6554959785522788

macro
Precision: 0.5327906150017775
Recall: 0.5996347589639599
F1 Score: 0.4908733656042393

weighted
Precision: 0.8748318158988
Recall: 0.6554959785522788
F1 Score: 0.7331891589204955

start DF
Accuracy: 0.6775818639798489

macro
Precision: 0.5630131362889984
Recall: 0

In [20]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(68, 56) (68,)
Accuracy: 0.9117647058823529

macro
Precision: 0.5841269841269842
Recall: 0.6358974358974359
F1 Score: 0.6015625

weighted
Precision: 0.9343604108309991
Recall: 0.9117647058823529
F1 Score: 0.9221047794117647

start RF 10 BIM
(895, 56) (895,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.32849162011173183

macro
Precision: 0.5
Recall: 0.16424581005586592
F1 Score: 0.24726661059714045

weighted
Precision: 1.0
Recall: 0.32849162011173183
F1 Score: 0.4945332211942809

start RF 10 FGSM
(862, 56) (862,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.728538283062645

macro
Precision: 0.5
Recall: 0.3642691415313225
F1 Score: 0.4214765100671141

weighted
Precision: 1.0
Recall: 0.728538283062645
F1 Score: 0.8429530201342282

start RF 10 PGD
(895, 56) (895,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.40730337078651685

macro
Precision: 0.5
Recall: 0.20365168539325842
F1 Score: 0.2894211576846307

weighted
Precision: 1.0
Recall: 0.40730337078651685
F1 Score: 0.5788423153692615

start RF 10 ZOO
(198, 56) (198,)
Accuracy: 0.9090909090909091

macro
Precision: 0.5836065573770491
Recall: 0.6349206349206349
F1 Score: 0.6008064516129032

weighted
Precision: 0.9323397913561848
Recall: 0.9090909090909091
F1 Score: 0.9197214076246335

start RF 10 CaFA
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 SINIFGSM
(1713, 56) (1713,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.29071803852889666

macro
Precision: 0.5
Recall: 0.14535901926444833
F1 Score: 0.22523744911804613

weighted
Precision: 1.0
Recall: 0.29071803852889666
F1 Score: 0.45047489823609227

start RF 10 VNIFGSM
(2193, 56) (2193,)
Accuracy: 0.3743730050159599

macro
Precision: 0.5
Recall: 0.18718650250797994
F1 Score: 0.2723954877239549

weighted
Precision: 1.0
Recall: 0.3743730050159599
F1 Score: 0.5447909754479098



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
epsilon = 0
Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(325, 56) (325,)
Accuracy: 0.963076923076923

macro
Precision: 0.9814241486068112
Recall: 0.5714285714285714
F1 Score: 0.6155362776025237

weighted
Precision: 0.9644486782567279
Recall: 0.963076923076923
F1 Score: 0.9495801989808298

start RF 10 BIM
(507, 56) (507,)
Accuracy: 0.9428007889546351

macro
Precision: 0.8243243243243243
Recall: 0.8970240700218819
F1 Score: 0.8555881861844754

weighted
Precision: 0.9508857970396432
Recall: 0.9428007889546351
F1 Score: 0.9456783621424829

start RF 10 FGSM
(439, 56) (439,)
Accuracy: 0.9658314350797267

macro
Precision: 0.5833333333333334
Recall: 0.9827981651376148
F1 Score: 0.6341056842807135

weighted
Precision: 0.9943052391799544
Recall: 0.9658314350797267
F1 Score: 0.9777354691993084

start RF 10 PGD
(507, 56) (507,)
Accuracy: 0.9428007889546351

macro
Precision: 0.8243243243243243
Recall: 0.8970240700218819
F1 Score: 0.8555881861844754

weighted
Precision: 0.9508857970396432
Recall: 0.9428007889546351
F1 Score: 0.945678

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9433962264150944

macro
Precision: 0.5
Recall: 0.4716981132075472
F1 Score: 0.4854368932038835

weighted
Precision: 1.0
Recall: 0.9433962264150944
F1 Score: 0.9708737864077671

start RF 10 AutoPGD
(914, 56) (914,)
Accuracy: 0.9879649890590809

macro
Precision: 0.9149345650500385
Recall: 0.9605238565668972
F1 Score: 0.9363940607463923

weighted
Precision: 0.9888577815286291
Recall: 0.9879649890590809
F1 Score: 0.9882782996806648

start RF 10 ZOO
(1331, 56) (1331,)
Accuracy: 0.9887302779864763

macro
Precision: 0.778208174904943
Recall: 0.762042259826305
F1 Score: 0.7698744769874477

weighted
Precision: 0.9884061484975133
Recall: 0.9887302779864763
F1 Score: 0.9885616679702982

start RF 10 CaFA
(530, 56) (530,)
Accuracy: 0.8943396226415095

macro
Precision: 0.8435927216415021
Recall: 0.9032098765432099
F1 Score: 0.8659221915035868

weighted
Precision: 0.9119058576030503
Recall: 0.8943396226415095
F1 Score: 0.8985323583831701

start RF 10 SINIFGSM
(472, 56) (472,)
Accuracy: 0.

In [22]:
epsilon = 0

Def = "Def4"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
Accuracy: 0.7199363732767763

macro
Precision: 0.7329848947966904
Recall: 0.7369276933940886
F1 Score: 0.7197082910596397

weighted
Precision: 0.7524857183181798
Recall: 0.7199363732767763
F1 Score: 0.7210869590338546

start RF 10 BIM
Accuracy: 0.8039558344586544

macro
Precision: 0.6682013526769125
Recall: 0.7268351438165676
F1 Score: 0.6868889204308908

weighted
Precision: 0.8420896431590617
Recall: 0.8039558344586544
F1 Score: 0.8183731169033328

start RF 10 FGSM
Accuracy: 0.8486856257798047

macro
Precision: 0.7973629716058097
Recall: 0.7554426419126854
F1 Score: 0.7726104023886358

weighted
Precision: 0.8415064900662234
Recall: 0.8486856257798047
F1 Score: 0.8429647056036316

start RF 10 PGD
Accuracy: 0.8039558344586544

macro
Precision: 0.6682013526769125
Recall: 0.7268351438165676
F1 Score: 0.6868889204308908

weighted
Precision: 0.8420896431590617
Recall: 0.8039558344586544
F1 Score: 0.8183731169033328

start RF 10 DF
Accuracy: 0.5071065989847716

macro
Pre

In [15]:

# torch.save(model.state_dict(), "./Free_Adversarial_Training.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")